In [1]:
# import relevant modules
import pandas as pd

# find csv and read as data frame
file = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file)

# display df for reference
purchase_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [2]:
# check data cleanliness
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [3]:
# find unique names under SN
# create an array of unique SNs
uniqueSN = purchase_data["SN"].unique()
# find the number of elements within this array
totalUsers = len(uniqueSN)


print("There are a total of " + str(totalUsers) + " players playing the game.")

There are a total of 576 players playing the game.


In [4]:
# complete purchasing analysis calculations
# number of unique items
uniqueItems = purchase_data["Item ID"].unique()
uniqueItemCount = len(uniqueItems)
# average purchase price
averagePurchasePrice = purchase_data["Price"].mean()
# total count of purchases
uniquePurchases = purchase_data["Purchase ID"].count()
# total revenue
totalRevenue = purchase_data["Price"].sum()

# display these results
purchaseAnalysis_df = pd.DataFrame(
    {"Unique Items Purchased": uniqueItemCount,
    "Average Purchase Price": [averagePurchasePrice],
    "Total Number of Purchases": uniquePurchases,
    "Total Revenue": totalRevenue})


purchaseAnalysis_df.style.hide_index().format({"Average Purchase Price": "${:.2f}", "Total Revenue": "${:.2f}"})

Unique Items Purchased,Average Purchase Price,Total Number of Purchases,Total Revenue
179,$3.05,780,$2379.77


In [5]:
# gender demographic analysis
# create df of only SN and gender
genderCount_df = purchase_data[["SN","Gender"]]
# drop duplicate SNs
genderCount_df = genderCount_df.drop_duplicates(subset="SN")
genderCount_df = genderCount_df.set_index("SN")

genderCount_df["Gender"].value_counts()

Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [6]:
# gender demographic analysis (cont'd)
# count total players, males, females, other
totalCount = len(genderCount_df)
maleCount = len(genderCount_df.loc[genderCount_df["Gender"] == "Male",:])
femaleCount = len(genderCount_df.loc[genderCount_df["Gender"] == "Female",:])
otherCount = len(genderCount_df.loc[genderCount_df["Gender"] == "Other / Non-Disclosed",:])

# create df for displaying the results
genderAnalysis_df = pd.DataFrame(
    {"Gender": ["Male","Female","Other/Non-Disclosed"],
    "Count": [maleCount, femaleCount, otherCount]})

# add column of percentages
genderAnalysis_df["Percentage of Players"] = ((genderAnalysis_df["Count"]/totalCount)*100)
genderAnalysis_df["Percentage of Players"] = genderAnalysis_df["Percentage of Players"].map("{:.2f}%".format)

# display the results
genderAnalysis_df.style.hide_index()

Gender,Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [7]:
# gender purchasing analysis
# begin by grouping information by gender
genderAnalysisGroups = purchase_data.groupby(["Gender"])

genderPurchaseCount = genderAnalysisGroups["Purchase ID"].count()
genderPurchaseAvgPrice = genderAnalysisGroups["Price"].mean()
genderTotalPurchaseValue = genderAnalysisGroups["Price"].sum()
# found easier way to count unique values in a dataframe here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html
genderAveragePerPerson = genderTotalPurchaseValue/genderAnalysisGroups["SN"].nunique()

genderGroupAnalysisSummary = pd.DataFrame({"Purchase Count": genderPurchaseCount,
                                          "Average Purchase Price": genderPurchaseAvgPrice,
                                          "Total Purchase Value": genderTotalPurchaseValue,
                                          "Average Total Purchase per Person": genderAveragePerPerson})

genderGroupAnalysisSummary["Average Purchase Price"] = genderGroupAnalysisSummary["Average Purchase Price"].map("${:.2f}".format)
genderGroupAnalysisSummary["Total Purchase Value"] = genderGroupAnalysisSummary["Total Purchase Value"].map("${:.2f}".format)
genderGroupAnalysisSummary["Average Total Purchase per Person"] = genderGroupAnalysisSummary["Average Total Purchase per Person"].map("${:.2f}".format)

genderGroupAnalysisSummary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [9]:
# age demographic analysis
# create bins and add column with binned age data
bins = [0,9.9,19.9,24.9,29.9,39.9,999]
group_names = ["<10","10-19","20-24","25-29","30-39","40+"]

purchase_data["Age Demographic"] = pd.cut(purchase_data["Age"], bins, labels=group_names, include_lowest=True)

# create df of only SN and age
ageCount_df = purchase_data[["SN","Age Demographic"]]
# drop duplicate SNs
ageCount_df = ageCount_df.drop_duplicates(subset="SN")
ageCount_df = ageCount_df.set_index("SN")

ageCount_df["Age Demographic"].value_counts()

20-24    258
10-19    129
30-39     83
25-29     77
<10       17
40+       12
Name: Age Demographic, dtype: int64

In [11]:
# age demographic analysis (cont'd)
# count total players, and total players in each of 6 bins
totalCount = len(ageCount_df)
bin1Count = len(ageCount_df.loc[ageCount_df["Age Demographic"] == "<10",:])
bin2Count = len(ageCount_df.loc[ageCount_df["Age Demographic"] == "10-19",:])
bin3Count = len(ageCount_df.loc[ageCount_df["Age Demographic"] == "20-24",:])
bin4Count = len(ageCount_df.loc[ageCount_df["Age Demographic"] == "25-29",:])
bin5Count = len(ageCount_df.loc[ageCount_df["Age Demographic"] == "30-39",:])
bin6Count = len(ageCount_df.loc[ageCount_df["Age Demographic"] == "40+",:])


# create df for displaying the results
ageAnalysis_df = pd.DataFrame(
    {"Age Group": group_names,
    "Count": [bin1Count,bin2Count,bin3Count,bin4Count,bin5Count,bin6Count]})

# add column of percentages
ageAnalysis_df["Percentage of Players"] = ((ageAnalysis_df["Count"]/totalCount)*100)
ageAnalysis_df["Percentage of Players"] = ageAnalysis_df["Percentage of Players"].map("{:.2f}%".format)

# display the results
ageAnalysis_df.style.hide_index()

Age Group,Count,Percentage of Players
<10,17,2.95%
10-19,129,22.40%
20-24,258,44.79%
25-29,77,13.37%
30-39,83,14.41%
40+,12,2.08%


In [13]:
# age purchasing analysis
# begin by grouping information by age
ageAnalysisGroups = purchase_data.groupby(["Age Demographic"])

agePurchaseCount = ageAnalysisGroups["Purchase ID"].count()
agePurchaseAvgPrice = ageAnalysisGroups["Price"].mean()
ageTotalPurchaseValue = ageAnalysisGroups["Price"].sum()
# found easier way to count unique values in a dataframe here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html
ageAveragePerPerson = ageTotalPurchaseValue/ageAnalysisGroups["SN"].nunique()

ageGroupAnalysisSummary = pd.DataFrame({"Purchase Count": agePurchaseCount,
                                          "Average Purchase Price": agePurchaseAvgPrice,
                                          "Total Purchase Value": ageTotalPurchaseValue,
                                          "Average Total Purchase per Person": ageAveragePerPerson})

ageGroupAnalysisSummary["Average Purchase Price"] = ageGroupAnalysisSummary["Average Purchase Price"].map("${:.2f}".format)
ageGroupAnalysisSummary["Total Purchase Value"] = ageGroupAnalysisSummary["Total Purchase Value"].map("${:.2f}".format)
ageGroupAnalysisSummary["Average Total Purchase per Person"] = ageGroupAnalysisSummary["Average Total Purchase per Person"].map("${:.2f}".format)

ageGroupAnalysisSummary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Demographic,,,,
<10,23,$3.35,$77.13,$4.54
10-19,164,$3.02,$495.67,$3.84
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-39,114,$3.17,$361.67,$4.36
40+,13,$2.94,$38.24,$3.19


In [18]:
# top spenders
topSpenders = purchase_data.groupby(["SN"])

topPurchaseCount = topSpenders["Purchase ID"].count()
topPurchaseAvgPrice = topSpenders["Price"].mean()
topTotalPurchaseValue = topSpenders["Price"].sum()

topSpenderAnalysis = pd.DataFrame({"Purchase Count": topPurchaseCount,
                                          "Average Purchase Price": topPurchaseAvgPrice,
                                          "Total Purchase Value": topTotalPurchaseValue})

topSpenderAnalysis_sorted = topSpenderAnalysis.sort_values("Total Purchase Value", ascending=False)

topSpenderAnalysis_sorted["Average Purchase Price"] = topSpenderAnalysis_sorted["Average Purchase Price"].map("${:.2f}".format)
topSpenderAnalysis_sorted["Total Purchase Value"] = topSpenderAnalysis_sorted["Total Purchase Value"].map("${:.2f}".format)

topSpenderAnalysis_sorted.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [26]:
# top items by popularity
topItems = purchase_data.groupby(["Item ID", "Item Name"])

itemPurchaseCount = topItems["Purchase ID"].count()
itemPurchaseAvgPrice = topItems["Price"].mean()
itemTotalPurchaseValue = topItems["Price"].sum()

topItemAnalysis = pd.DataFrame({"Purchase Count": itemPurchaseCount,
                                "Average Purchase Price": itemPurchaseAvgPrice,
                                "Total Purchase Value": itemTotalPurchaseValue})

topItemAnalysis_sorted = topItemAnalysis.sort_values("Purchase Count", ascending=False)

topItemAnalysis_sorted["Average Purchase Price"] = topItemAnalysis_sorted["Average Purchase Price"].map("${:.2f}".format)
topItemAnalysis_sorted["Total Purchase Value"] = topItemAnalysis_sorted["Total Purchase Value"].map("${:.2f}".format)

topItemAnalysis_sorted.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [27]:
# top items by popularity
topItemAnalysis_sorted_profit = topItemAnalysis.sort_values("Total Purchase Value", ascending=False)

topItemAnalysis_sorted_profit["Average Purchase Price"] = topItemAnalysis_sorted_profit["Average Purchase Price"].map("${:.2f}".format)
topItemAnalysis_sorted_profit["Total Purchase Value"] = topItemAnalysis_sorted_profit["Total Purchase Value"].map("${:.2f}".format)

topItemAnalysis_sorted_profit.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
